In [13]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import collections
import matplotlib.pyplot as plt
# data process
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
all_data= pd.read_excel("./20160809sakujo.xlsx")




#print(all_data.head())
#sys.exit()
#print(all_data_2.head())
#print(all_data.isnull().sum())
all_data_2 = all_data
#all_data_2 de,zyuuhukusurugyougaaru
#(all_data_2[all_data_2.duplicated()])

#sys.exit()
#print(all_data_2.isnull().sum())
#print(all_data_2.describe())
#sys.exit()
#print(len(all_data2))
time = []
time = all_data_2["Time"].unique()
kyoten = []
kyoten = all_data_2["name"].unique()

#band = []
list2 = []
list3 = []
list4 = []
pixel2 = []
CH2 = []
Cloud = []
terrian = []
CC = []
CS = []
for t in time:
    for n in kyoten:
        #band = all_data_2["band"][(all_data_2["Time"] == t) & (all_data_2["name"]== n)].values
        pixel2 = all_data_2["pixel"][(all_data_2["Time"] == t) & (all_data_2["name"] == n)].values
        CH2 = all_data_2["CH"][(all_data_2["Time"] == t) & (all_data_2["name"] == n)].values
        Cloud = all_data_2["cloud"][(all_data_2["Time"] == t) & (all_data_2["name"] == n)].values
        #terrian = all_data_2["terrian"][(all_data_2["Time"] == t) & (all_data_2["name"] == n)].values
        CC = all_data_2["cloudconfidence"][(all_data_2["Time"] == t) & (all_data_2["name"] == n)].values
        CS = all_data_2["cloudshade"][(all_data_2["Time"] == t) & (all_data_2["name"] == n)].values
        # print(CH1)
        if len(CH2) != 0:
            #list2 += [[n,t,pixel2, Cloud[0], terrian[0], CC[0], CS[0], CH2[0]]]
            list3 += [[pixel2, Cloud[0],CC[0], CS[0]]]
            list4 += [[CH2[0]]]

# print(list)

# array = np.array(list)
# rray
#array8 = np.array(list2)
#array8

""" 
with open("/home/meinai929/data_cleansed/mistake.txt","w") as f: 
    for i in range(0,1134): 
    #print(len(data_array[i][0])) 
        if len(array8[i][2]) != 11: 
            print(array8[i],file = f) 
 
"""
#ys.exit()
data_array = np.array(list3)
CH_array = np.array(list4)

#print(data_array.shape)
#print(CH_array.shape)
#sys.exit()
#data_array & CH_array numuber of rows are same (1134,1) (1134,5)
#print(data_array[0][0])
#print(len(data_array[0][0]))

#band no kazu ga, 10 to 22 no tokoro ga aru


#np.savetxt("/home/meinai929/data_cleansed/data_array_1205.txt",data_array,fmt = "%s")

#sys.exit()
def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

list1 =[]
for x in data_array:
    list1 += flatten(x)

#print(len(list1))
#26415 1761*15= 26415ninaruhazu
#sys.exit()

def split_list(l, n):
    """ 
    リストをサブリストに分割する 
    :param l: リスト 
    :param n: サブリストの要素数 
    :return: 
    """
    for idx in range(0, len(l), n):
        yield l[idx:idx + n]

#print(list(split_list(list1, 14)))
inputs = np.array(list(split_list(list1, 14)))


scaler = preprocessing.MinMaxScaler()
sc = StandardScaler()
#inputs_seikika = scaler.fit_transform(inputs)
inputs_seikika_pixel = scaler.fit_transform(inputs[:,0:11])
inputs_seikika_cloud = scaler.fit_transform(inputs[:,11:])
#pixel_seikika = sc.fit_transform(inputs[:,0:11])
#cloud_seikika = sc.fit_transform(inputs[:,11:])
#print(inputs_seikika_pixel)
#print(inputs_seikika_cloud)

#nputs_seikika = np.concatenate((inputs_seikika_pixel,inputs_seikika_cloud),axis = 1)

#seikika = np.concatenate((pixel_seikika,cloud_seikika),axis=1)
seikika = np.concatenate((inputs_seikika_pixel,inputs_seikika_cloud),axis=1)
#print(inputs_seikika)


In [17]:
from sklearn.model_selection import train_test_split
CH_seikika = scaler.fit_transform(CH_array)
#CH_seikika = sc.fit_transform(CH_array)
#print(CH_seikika)
#sys.exit()
#plt.plot(inputs_seikika,CH_array)
#plt.show()
#print(CH_seikika)
#sys.exit()

#print(inputs_seikika)

#sys.exit()
inputs_train, inputs_test = train_test_split(seikika, shuffle=False)
CH_train, CH_test = train_test_split(CH_seikika, shuffle=False)

#print(inputs_train.shape)
#sys.exit()
#print(inputs_train[0])
#print(CH_train.shape)
#print(inputs_train.shape)
#print(inputs.shape)
#print(CH_train.shape)
#sys.exit()
#print(inputs_train,CH_train)
#print(inputs_train.shape)
#print(inputs_train)

#sys.exit()
#input 15 * 1134
#model kouchiku
inputs = keras.Input(shape =(14,) )

dense = layers.Dense(64,activation ="tanh")
x = dense(inputs)

x = layers.Dense(64,activation="tanh")(x)

outputs = layers.Dense(1)(x)

model = keras.Model(inputs=inputs,outputs=outputs,name="test1")

model.compile(
    loss="mean_squared_error",
    optimizer="sgd",
    #metrics=["accuracy"],
    metrics=[tf.keras.metrics.MeanAbsolutePercentageError(
    name="mean_absolute_percentage_error", dtype=None
)]
)

history = model.fit(inputs_train, CH_train, batch_size=100, epochs=100, validation_split=0.2)

test_scores = model.evaluate(inputs_test, CH_test, verbose=2)
print("Test loss:", test_scores[0])
print("MAPE:", test_scores[1])

Train on 1004 samples, validate on 251 samples
Epoch 1/100
1004/1004 [==============================] - 1s 609us/sample - loss: 0.0652 - mean_absolute_percentage_error: 46244.1797 - val_loss: 0.0252 - val_mean_absolute_percentage_error: 212.7171
Epoch 2/100
1004/1004 [==============================] - 0s 29us/sample - loss: 0.0228 - mean_absolute_percentage_error: 69999.7266 - val_loss: 0.0141 - val_mean_absolute_percentage_error: 157.6517
Epoch 3/100
1004/1004 [==============================] - 0s 29us/sample - loss: 0.0153 - mean_absolute_percentage_error: 24860.5703 - val_loss: 0.0112 - val_mean_absolute_percentage_error: 136.9294
Epoch 4/100
1004/1004 [==============================] - 0s 30us/sample - loss: 0.0126 - mean_absolute_percentage_error: 39544.2500 - val_loss: 0.0101 - val_mean_absolute_percentage_error: 128.7227
Epoch 5/100
1004/1004 [==============================] - 0s 30us/sample - loss: 0.0113 - mean_absolute_percentage_error: 43569.8047 - val_loss: 0.0111 - val_mea

In [15]:
print(CH_seikika)

[[0.05168444]
 [0.05031091]
 [0.05443151]
 ...
 [0.11776621]
 [0.03909823]
 [0.09800659]]
